In [1]:
import sklearn
sklearn.__version__

'0.24.1'

In [ ]:
!pip -q uninstall scikit-learn -y
!pip -q install -U scikit-learn

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
cd /content/drive/My\ Drive/FYP/Genomic

/content/drive/.shortcut-targets-by-id/1aCPQAeC2tHBLx06_jt8sdd_-fseDZRw7/FYP/Genomic


In [4]:
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

data = pd.read_excel("Clean genomic dataset.xlsx")

In [5]:
data.dtypes

Virus name                               object
Virus family                             object
DNA(0)/RNA(1)                             int64
Virus genus                              object
Average genome length (Nucleotides)       int64
Replication in the cytoplasm (True:1)     int64
Vector-borne or not (True:1)              int64
enveloped/non-enveloped (True:1)          int64
vSegmentedTF (True:1)                     int64
Zoonotic or not (True:1)                  int64
dtype: object

<h2 id="load_dataset">Virus Zoonotic Potential Model (1 for Zoonotic, 0 for non Zoonotic) (AdaBoost)</h2>

In [6]:
data

,Virus name,Virus family,DNA(0)/RNA(1),Virus genus,Average genome length (Nucleotides),Replication in the cytoplasm (True:1),Vector-borne or not (True:1),enveloped/non-enveloped (True:1),vSegmentedTF (True:1),Zoonotic or not (True:1)
0,Adelaide_River_virus,Rhabdoviridae,1,Ephemerovirus,14627,1,1,0,0,0
1,African_green_monkey_polyomavirus,Polyomaviridae,0,Polyomavirus,5270,1,0,1,0,1
2,African_green_monkey_simian_foamy_virus,Retroviridae,1,Spumavirus,13111,0,0,0,0,0
3,African_horse_sickness_virus,Reoviridae,1,Orbivirus,19200,1,1,1,1,0
4,African_swine_fever_virus,Asfarviridae,0,Asfivirus,170101,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...
540,Zaire_ebolavirus,Filoviridae,1,Ebolavirus,18898,1,0,0,0,1
541,Zika_virus,Flaviviridae,1,Flavivirus,10794,1,1,0,0,1
542,Sars-cov-2,Coronaviridae,1,Betacoronavirus,29811,1,0,0,0,1
543,H1N1,Orthomyxovirida,1,Alphainfluenzavirus,2341,1,0,1,1,1


In [7]:
data['Virus family'] = LabelEncoder().fit_transform(data['Virus family'])
X = data[['Virus family','vSegmentedTF (True:1)', 'Vector-borne or not (True:1)', 'enveloped/non-enveloped (True:1)', 
          'DNA(0)/RNA(1)', 'Replication in the cytoplasm (True:1)','Average genome length (Nucleotides)']].values
Y = data['Zoonotic or not (True:1)'].values

In [8]:
from sklearn import preprocessing
X= preprocessing.StandardScaler().fit(X).transform(X)
# Y= preprocessing.StandardScaler().fit(Y).transform(X)


In [9]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=1)

In [10]:
x_train[0:5]

array([[ 0.33350136, -0.57381904, -0.64180768,  1.55809915, -1.40646439,
        -1.41811485, -0.53569016],
       [ 1.56529669, -0.57381904,  1.55809915, -0.64180768,  0.71100272,
         0.7051615 , -0.46647866],
       [ 1.42843054, -0.57381904, -0.64180768, -0.64180768,  0.71100272,
         0.7051615 , -0.46163368],
       [ 0.33350136, -0.57381904, -0.64180768,  1.55809915, -1.40646439,
        -1.41811485, -0.53070525],
       [ 0.60723366, -0.57381904, -0.64180768,  1.55809915, -1.40646439,
        -1.41811485, -0.58120149]])

In [11]:
y_train[0:5]

array([0, 1, 0, 0, 0])

In [12]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score, roc_auc_score

In [13]:
M1 = AdaBoostClassifier(n_estimators=50,algorithm='SAMME').fit(x_train,y_train)
M1

AdaBoostClassifier(algorithm='SAMME')

In [14]:
kf = KFold(n_splits=6, random_state=None, shuffle=False)
i = 0
cum_ROC, cum_acc, cum_F1 = 0, 0, 0

for train_index, test_index in kf.split(X):

    x_train, x_test = X[train_index], X[test_index]
    y_train, y_test = Y[train_index], Y[test_index]
    M1 = AdaBoostClassifier(n_estimators=50,algorithm='SAMME').fit(x_train, y_train)
    yhat = M1.predict(x_test)

    # print("Test set ROC_AUC is: ", roc_auc_score(y_test, yhat))
    # print("Test set Accuracy: ", accuracy_score(y_test, yhat))
    # print('The F1 score is', f1_score(y_test, yhat, average='weighted'))
    # print('\n')

    cum_ROC += roc_auc_score(y_test, yhat)
    cum_acc += accuracy_score(y_test, yhat)
    # cum_F1  += f1_score(y_test, yhat)

    i+=1


print("Test set ROC_AUC is: ", cum_ROC/i)
print("Test set Accuracy: ", cum_acc/i)
print('The F1 score is', cum_F1/i)

Test set ROC_AUC is:  0.5617154964069985
Test set Accuracy:  0.6548026048026049
The F1 score is 0.0


In [15]:
kf = KFold(n_splits=6, random_state=None, shuffle=False)
idx = 0
cum_ROC, cum_acc, cum_F1 = 0, 0, 0

for train_index, test_index in kf.split(X):
    x_train, x_test = X[train_index], X[test_index]
    y_train, y_test = Y[train_index], Y[test_index]

    i = tf.keras.layers.Input(shape=x_train.shape[1])

    x = tf.keras.layers.Dense(64, activation='relu')(i)

    x = tf.keras.layers.Dense(32, activation='relu')(x)

    y = tf.keras.layers.Dense(1, activation='sigmoid')(x)

    mlp = tf.keras.models.Model(i, y, name='mlp')

    mlp.compile('adam', 'binary_crossentropy', metrics=['accuracy', tf.keras.metrics.AUC()])

    mlp.fit(x_train, y_train, batch_size=16, epochs=30, verbose=0)
    
    yhat = mlp.predict(x_test)
    yhat = np.where(yhat>0.5, 1, 0)

    cum_ROC += roc_auc_score(y_test, yhat)
    cum_acc += accuracy_score(y_test, yhat)
    # cum_F1  += f1_score(y_test, yhat)

    idx+=1

print("Test set ROC_AUC is: ", cum_ROC/idx)
print("Test set Accuracy: ", cum_acc/idx)
print('The F1 score is', cum_F1/idx)

Test set ROC_AUC is:  0.6019643019384358
Test set Accuracy:  0.6733821733821735
The F1 score is 0.0


In [17]:
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.metrics import jaccard_score
from sklearn.metrics import f1_score

In [18]:
yhat=M1.predict(x_test)

print("Test set ROC_AUC is: ", roc_auc_score(y_test, yhat))
print("Test set Accuracy: ", accuracy_score(y_test, yhat))
print('The F1 score is', f1_score(y_test, yhat, average='weighted'))

Test set ROC_AUC is:  0.5113636363636364
Test set Accuracy:  0.5222222222222223
The F1 score is 0.3700411522633745


<h2 id="load_dataset">Virus Class Model (1 for High Risk, 0 for Low Risk) (KMeans)</h2>

In [19]:
from sklearn import preprocessing
X= preprocessing.StandardScaler().fit(X).transform(X)
X[0:5]

array([[ 1.42843054, -0.57381904,  1.55809915, -0.64180768,  0.71100272,
         0.7051615 , -0.41533527],
       [ 0.88096595, -0.57381904, -0.64180768,  1.55809915, -1.40646439,
         0.7051615 , -0.57899763],
       [ 1.29156439, -0.57381904, -0.64180768, -0.64180768,  0.71100272,
        -1.41811485, -0.44185147],
       [ 1.15469824,  1.74270968,  1.55809915,  1.55809915,  0.71100272,
         0.7051615 , -0.33534937],
       [-1.44575855, -0.57381904,  1.55809915, -0.64180768, -1.40646439,
         0.7051615 ,  2.30404522]])

In [20]:
from sklearn.cluster import KMeans 

In [21]:
M2 = KMeans(init = "k-means++",n_clusters = 2 , n_init = 12,n_jobs=-1).fit(X)
M2


/usr/local/lib/python3.7/dist-packages/sklearn/cluster/_kmeans.py:793: FutureWarning: 'n_jobs' was deprecated in version 0.23 and will be removed in 1.0 (renaming of 0.25).
  " removed in 1.0 (renaming of 0.25).", FutureWarning)


KMeans(n_clusters=2, n_init=12, n_jobs=-1)

<h1 align=center><font size="4"> Now, these two Supervised and Unsupervised models' weights (M1 in which 1 is for zoonotic and M2 in which 1 is for high risk respectively) are to deployed onto our web portal </font></h1>

In [22]:
import pickle
filename = 'zoonotic_model.sav'
pickle.dump(M1, open(filename, 'wb'))

filename = 'risk_model.sav'
pickle.dump(M2, open(filename, 'wb'))

In [ ]:
import joblib
save_model = joblib.load('risk_model.sav')
x=save_model.predict([[ 1.42843054, -0.57381904,  1.55809915, -0.64180768,  0.71100272,0.7051615 , -0.41533527],[ 0.88096595, -0.57381904, -0.64180768,  1.55809915, -1.40646439, 0.7051615 , -0.57899763]])
# x = ['Adelaide_River_virus',	'Rhabdoviridae'	,1	,'Ephemerovirus'	,14627,	1,	1	,0,	0]
print(x)

[0 1]


In [ ]:
import joblib
save_model = joblib.load('risk_model.sav')
x=save_model.predict(x_train[0:5])
print(x)

[1 1 1 1 1]
